In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [50]:


# Set random seed for reproducibility
np.random.seed(42)

# Generate base data
def generate_inventory_data(num_records=1000):
    # Define possible values
    warehouse_ids = list(range(1, 6))  # 5 warehouses
    item_types = ['row', 'product']
    region_ids = list(range(1, 5))  # 4 regions
    
    # Generate random combinations
    data = {
        'timestamp': [],
        'warehouseid': [],
        'itemtype': [],
        'itemid': [],
        'regionid': [],
        'currentstock': []
    }
    
    # Create base patterns for each item
    num_items = 50
    base_stocks = {i: np.random.randint(50, 200) for i in range(1, num_items + 1)}
    
    # Generate time series data
    start_date = datetime(2023, 1, 1)
    
    for i in range(num_records):
        # Generate timestamp with daily frequency
        current_date = start_date + timedelta(days=i % 365)
        
        # Generate random warehouse and item information
        warehouse_id = np.random.choice(warehouse_ids)
        item_type = np.random.choice(item_types)
        item_id = np.random.randint(1, num_items + 1)
        region_id = np.random.choice(region_ids)
        
        # Generate stock levels with patterns
        base_stock = base_stocks[item_id]
        
        # Add seasonal pattern
        seasonal_factor = 1 + 0.3 * np.sin(2 * np.pi * (current_date.timetuple().tm_yday / 365))
        
        # Add trend
        trend = 0.1 * (i % 365) / 365
        
        # Add random variation
        noise = np.random.normal(0, 0.1)
        
        # Calculate final stock
        stock = int(max(0, base_stock * seasonal_factor * (1 + trend) * (1 + noise)))
        
        # Append to data dictionary
        data['timestamp'].append(current_date)
        data['warehouseid'].append(warehouse_id)
        data['itemtype'].append(item_type)
        data['itemid'].append(item_id)
        data['regionid'].append(region_id)
        data['currentstock'].append(stock)
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Sort by timestamp and warehouse/item information
    df = df.sort_values(['timestamp', 'warehouseid', 'itemid'])
    
    return df

In [51]:
# Generate the data
df = generate_inventory_data(5000)

In [52]:
# Add some patterns and relationships
def add_patterns(df):
    # Add weekend effect (lower stock on weekends)
    df['is_weekend'] = df['timestamp'].dt.weekday >= 5
    df.loc[df['is_weekend'], 'currentstock'] = df.loc[df['is_weekend'], 'currentstock'] * 0.9
    
    # Add regional patterns
    region_multipliers = {1: 1.2, 2: 0.9, 3: 1.1, 4: 0.8}
    for region, multiplier in region_multipliers.items():
        df.loc[df['regionid'] == region, 'currentstock'] = \
            df.loc[df['regionid'] == region, 'currentstock'] * multiplier
    
    # Round stock values to integers
    df['currentstock'] = df['currentstock'].astype(int)
    
    # Drop temporary columns
    df = df.drop('is_weekend', axis=1)
    
    return df

In [53]:
# Apply patterns and save final dataset
final_df = add_patterns(df)

In [54]:
# Example of how to prepare data for LSTM
def prepare_lstm_data(df, sequence_length=7):
    # Create sequences of data
    sequences = []
    targets = []
    
    # Group by warehouse and item
    for (warehouse, item), group in df.groupby(['warehouseid', 'itemid']):
        stock_values = group['currentstock'].values
        
        for i in range(len(stock_values) - sequence_length):
            sequences.append(stock_values[i:i+sequence_length])
            targets.append(stock_values[i+sequence_length])
    
    return np.array(sequences), np.array(targets)

In [55]:
# Example usage:
X, y = prepare_lstm_data(final_df)

# Save to CSV
final_df.to_csv('warehouse_inventory_data.csv', index=False)

print("Dataset shape:", final_df.shape)
print("\nSample of the generated data:")
print(final_df.head())
print("\nData statistics:")
print(final_df.describe())

Dataset shape: (5000, 6)

Sample of the generated data:
      timestamp  warehouseid itemtype  itemid  regionid  currentstock
730  2023-01-01            1      row      48         3           166
0    2023-01-01            1  product      50         4            57
2555 2023-01-01            2  product       5         3           111
3285 2023-01-01            2  product       9         1           159
4015 2023-01-01            2      row      20         1           210

Data statistics:
                 timestamp  warehouseid       itemid    regionid  currentstock
count                 5000  5000.000000  5000.000000  5000.00000   5000.000000
mean   2023-06-29 04:40:48     2.978600    25.571400     2.50600    124.004200
min    2023-01-01 00:00:00     1.000000     1.000000     1.00000     28.000000
25%    2023-03-31 00:00:00     2.000000    13.000000     1.00000     82.000000
50%    2023-06-28 00:00:00     3.000000    26.000000     3.00000    117.000000
75%    2023-09-26 00:00:00     4

In [26]:
# 1. Data Preparation with Chronological Order and Embeddings
def prepare_data(file_path, sequence_length=7):
    """
    Reads the CSV, extracts time features, encodes categorical variables (without scaling),
    scales continuous time features, and creates sequences.
    
    Categorical features: warehouseid, itemtype, itemid, regionid
    Continuous features: day_of_week, month
    """
    # Read and sort by time
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')
    
    # Extract additional time features
    df['day_of_week'] = df['timestamp'].dt.dayofweek  # 0 (Monday) to 6 (Sunday)
    df['month'] = df['timestamp'].dt.month           # 1 to 12

    # Define which columns are categorical vs. continuous
    categorical_cols = ['warehouseid', 'itemtype', 'itemid', 'regionid']
    continuous_cols = ['day_of_week', 'month']
    
    # Label-encode categorical variables and capture vocabulary sizes for embeddings
    cat_vocab_sizes = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        cat_vocab_sizes[col] = df[col].nunique()
    
    # Scale continuous features
    scaler = MinMaxScaler()
    df[continuous_cols] = scaler.fit_transform(df[continuous_cols])
    
    # Create sequences – here we group by keys that define a unique time series.
    # (Adjust group_keys as per your domain; here we assume warehouseid & itemid identify a series.)
    X_cat, X_cont, y = [], [], []
    group_keys = ['warehouseid', 'itemid']
    
    for _, group in df.groupby(group_keys):
        group = group.sort_values('timestamp')
        # Extract values for categorical and continuous features
        cat_data = group[categorical_cols].values  # shape: (group_length, num_cat)
        cont_data = group[continuous_cols].values    # shape: (group_length, num_cont)
        target = group['currentstock'].values
        
        # Build sliding windows
        for i in range(len(group) - sequence_length):
            X_cat.append(cat_data[i:i+sequence_length])
            X_cont.append(cont_data[i:i+sequence_length])
            y.append(target[i+sequence_length])
    
    X_cat = np.array(X_cat)   # shape: (samples, sequence_length, num_cat)
    X_cont = np.array(X_cont) # shape: (samples, sequence_length, num_cont)
    y = np.array(y)
    
    return X_cat, X_cont, y, cat_vocab_sizes, scaler

In [27]:
# 2. Chronological Train/Validation/Test Split
def split_data(X_cat, X_cont, y, train_ratio=0.6, val_ratio=0.2):
    """
    Splits the data chronologically into train, validation, and test sets.
    For example, with 60% training, 20% validation, and 20% testing.
    """
    total = len(y)
    train_end = int(total * train_ratio)
    val_end = int(total * (train_ratio + val_ratio))
    
    X_cat_train = X_cat[:train_end]
    X_cont_train = X_cont[:train_end]
    y_train = y[:train_end]
    
    X_cat_val = X_cat[train_end:val_end]
    X_cont_val = X_cont[train_end:val_end]
    y_val = y[train_end:val_end]
    
    X_cat_test = X_cat[val_end:]
    X_cont_test = X_cont[val_end:]
    y_test = y[val_end:]
    
    return (X_cat_train, X_cont_train, y_train), (X_cat_val, X_cont_val, y_val), (X_cat_test, X_cont_test, y_test)


In [28]:
# 3. Model Definition with Embeddings, LSTM Layers, and Regularization
def create_model(sequence_length, num_cont_features, cat_vocab_sizes, embedding_dims=None):
    """
    Creates a multi-input LSTM model.
    For each categorical feature, an embedding layer is created.
    
    Args:
        sequence_length (int): Number of time steps per sample.
        num_cont_features (int): Number of continuous features.
        cat_vocab_sizes (dict): Mapping from categorical column name to vocabulary size.
        embedding_dims (dict): (Optional) Mapping from categorical column to embedding dimension.
                               If not provided, a default rule is used.
    """
    if embedding_dims is None:
        embedding_dims = {}
        # Simple rule: embedding dimension = min(50, (vocab_size + 1) // 2)
        for col, vocab_size in cat_vocab_sizes.items():
            embedding_dims[col] = min(50, (vocab_size + 1) // 2)
    
    categorical_cols = list(cat_vocab_sizes.keys())
    
    # Define an input for each categorical feature.
    cat_inputs = {}
    cat_embeddings = []
    for col in categorical_cols:
        inp = Input(shape=(sequence_length,), name=f'{col}_input')
        cat_inputs[col] = inp
        vocab_size = cat_vocab_sizes[col]
        embed_dim = embedding_dims[col]
        # Create an embedding layer; note that mask_zero is left as False assuming 0 is a valid index.
        embed = Embedding(input_dim=vocab_size, output_dim=embed_dim, name=f'{col}_embed')(inp)
        cat_embeddings.append(embed)
    
    # Concatenate embeddings along the last dimension.
    if len(cat_embeddings) > 1:
        cat_concat = Concatenate(name='cat_concat')(cat_embeddings)
    else:
        cat_concat = cat_embeddings[0]
    
    # Input layer for continuous features
    cont_input = Input(shape=(sequence_length, num_cont_features), name='cont_input')
    
    # Combine categorical and continuous features along the feature axis
    x = Concatenate(name='combined_features')([cat_concat, cont_input])
    
    # LSTM layers with bidirectionality, dropout, batch normalization, and L2 regularization.
    x = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, 
                           kernel_regularizer=tf.keras.regularizers.l2(1e-4)))(x)
    x = BatchNormalization()(x)
    
    x = Bidirectional(LSTM(64, activation='tanh', return_sequences=True, 
                           kernel_regularizer=tf.keras.regularizers.l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = LSTM(32, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = BatchNormalization()(x)
    
    # Fully connected layers for final feature extraction
    x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    
    output = Dense(1, name='output')(x)
    
    # Combine all inputs: categorical inputs (as a list) and continuous input.
    inputs = list(cat_inputs.values()) + [cont_input]
    model = Model(inputs=inputs, outputs=output)
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer,
                  loss='huber',  # More robust to outliers
                  metrics=['mae', 'mse'])
    
    return model

In [29]:
# 4. Training Function (No Shuffling for Chronology)
def train_model(model, train_data, val_data, epochs=50, batch_size=32):
    """
    Trains the model using the provided training and validation data.
    Note: `shuffle=False` is critical for chronological data.
    """
    X_cat_train, X_cont_train, y_train = train_data
    X_cat_val, X_cont_val, y_val = val_data
    
    # Prepare the input dictionary. The order of inputs must match the model inputs.
    # Here we assume the categorical columns are in the order: 
    # ['warehouseid', 'itemtype', 'itemid', 'regionid'].
    input_train = {
        'warehouseid_input': X_cat_train[:, :, 0],
        'itemtype_input': X_cat_train[:, :, 1],
        'itemid_input': X_cat_train[:, :, 2],
        'regionid_input': X_cat_train[:, :, 3],
        'cont_input': X_cont_train
    }
    input_val = {
        'warehouseid_input': X_cat_val[:, :, 0],
        'itemtype_input': X_cat_val[:, :, 1],
        'itemid_input': X_cat_val[:, :, 2],
        'regionid_input': X_cat_val[:, :, 3],
        'cont_input': X_cont_val
    }
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, mode='min')
    ]
    
    history = model.fit(
        input_train, y_train,
        validation_data=(input_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        shuffle=False,  # Do not shuffle – we must keep the chronological order!
        verbose=1
    )
    
    return history

In [30]:
# 5. Main Execution: Data Prep, Splitting, Model Creation, Training, and Evaluation
sequence_length = 7  # Number of past time steps used for prediction
file_path = 'data/warehouse_inventory_data.csv'
    
# Prepare the data
X_cat, X_cont, y, cat_vocab_sizes, cont_scaler = prepare_data(file_path, sequence_length)
    
# Chronologically split the data into train (60%), validation (20%), and test (20%) sets.
train_data, val_data, test_data = split_data(X_cat, X_cont, y, train_ratio=0.6, val_ratio=0.2)
    
# Build the model.
num_cont_features = X_cont.shape[2]  # e.g., 2 for day_of_week and month.
model = create_model(sequence_length, num_cont_features, cat_vocab_sizes)
model.summary()
    
# Train the model (no shuffling for time series!)
history = train_model(model, train_data, val_data, epochs=50, batch_size=32)
    
# Prepare test input dictionary
X_cat_test, X_cont_test, y_test = test_data
test_inputs = {
    'warehouseid_input': X_cat_test[:, :, 0],
    'itemtype_input': X_cat_test[:, :, 1],
    'itemid_input': X_cat_test[:, :, 2],
    'regionid_input': X_cat_test[:, :, 3],
    'cont_input': X_cont_test
}
    
# Evaluate the model on the test set.
test_loss, test_mae, test_mse = model.evaluate(test_inputs, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.2f}, Test MAE: {test_mae:.2f}, Test MSE: {test_mse:.2f}")
    
# Make predictions on the test set.
predictions = model.predict(test_inputs)
print("Predictions shape:", predictions.shape)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 warehouseid_input (InputLayer)  [(None, 7)]         0           []                               
                                                                                                  
 itemtype_input (InputLayer)    [(None, 7)]          0           []                               
                                                                                                  
 itemid_input (InputLayer)      [(None, 7)]          0           []                               
                                                                                                  
 regionid_input (InputLayer)    [(None, 7)]          0           []                               
                                                                                            

In [31]:
# --- Testing on a new dataset ---
# If you have a separate CSV for new data, process it with the same function.
new_file_path = 'data/test_data_inventory.csv'  # Update this path to your new data file
# Process new data
X_cat_new, X_cont_new, y_new, _, _ = prepare_data(new_file_path, sequence_length)

# Debug: print the shape of X_cat_new
print("Shape of X_cat_new:", X_cat_new.shape)

# Check if the new data produced valid sequences
if X_cat_new.ndim != 3 or X_cat_new.shape[0] == 0:
    raise ValueError("Insufficient data in the new CSV file to create sequences. "
                     "Ensure that the file has enough rows for at least one sequence of length "
                     f"{sequence_length} for each group.")

# Build input dictionary for the new test data.
new_test_inputs = {
    'warehouseid_input': X_cat_new[:, :, 0],
    'itemtype_input': X_cat_new[:, :, 1],
    'itemid_input': X_cat_new[:, :, 2],
    'regionid_input': X_cat_new[:, :, 3],
    'cont_input': X_cont_new
}

# Evaluate the model on the new data.
new_loss, new_mae, new_mse = model.evaluate(new_test_inputs, y_new, verbose=1)
print(f"\nNew Data - Loss: {new_loss:.2f}, MAE: {new_mae:.2f}, MSE: {new_mse:.2f}")

# Make predictions on the new data.
new_predictions = model.predict(new_test_inputs)
print("Predictions shape on new data:", new_predictions.shape)


Shape of X_cat_new: (9, 7, 4)
1/1 [==============================] - 0s 190ms/step - loss: 71.2700 - mae: 71.6027 - mse: 8216.3867

New Data - Loss: 71.27, MAE: 71.60, MSE: 8216.39
1/1 [==============================] - 0s 250ms/step
Predictions shape on new data: (9, 1)


In [32]:
current_sequence = X_test.copy()
current_sequence

array([[[7.50000000e-01, 1.00000000e+00, 6.66666667e-01, 6.66666667e-01,
         2.72727273e-01, 1.25000000e+02],
        [7.50000000e-01, 1.00000000e+00, 6.66666667e-01, 0.00000000e+00,
         3.63636364e-01, 1.06000000e+02],
        [7.50000000e-01, 1.00000000e+00, 3.33333333e-01, 3.33333333e-01,
         4.54545455e-01, 9.20000000e+01],
        ...,
        [7.50000000e-01, 1.00000000e+00, 6.66666667e-01, 8.33333333e-01,
         9.09090909e-01, 6.60000000e+01],
        [7.50000000e-01, 1.00000000e+00, 3.33333333e-01, 6.66666667e-01,
         9.09090909e-01, 6.70000000e+01],
        [7.50000000e-01, 1.00000000e+00, 1.00000000e+00, 6.66666667e-01,
         1.00000000e+00, 6.40000000e+01]],

       [[1.00000000e+00, 0.00000000e+00, 3.33333333e-01, 0.00000000e+00,
         0.00000000e+00, 1.44000000e+02],
        [1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 8.33333333e-01,
         0.00000000e+00, 1.35000000e+02],
        [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.333333

In [33]:
# Example prediction
sample_sequence = X_test[0]
future_predictions = predict_future_stock(model, sample_sequence)
print("\nNext 7 days stock predictions for a sample item:")
print(future_predictions)

NameError: name 'predict_future_stock' is not defined